<a href="https://colab.research.google.com/github/Medynal/Pollution/blob/main/pollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!git clone https://github.com/Medynal/Pollution.git

Cloning into 'Pollution'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 102 (delta 34), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (102/102), 2.12 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [87]:
%cd Pollution

/root/Pollution/Pollution/Pollution/Pollution


In [90]:
%ls

data_folder/        Jaipur_data.csv      Lucknow_data.csv  README.md
Gurugram_data.csv   Jorapokhar_data.csv  Mumbai_data.csv   Shillong_data.csv
Guwahati_data.csv   Kochi_data.csv       Patna_data.csv
Hyderabad_data.csv  Kolkata_data.csv     pollution.ipynb


In [84]:
# importing the necessary libraries for data analysis
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [91]:
pollution_dataset= '/content/Pollution/data_folder'
dataframes = []
for filename in os.listdir(pollution_dataset):
    if filename.endswith('.csv'):  # Check if the file is a CSV file
        file_path = os.path.join(pollution_dataset, filename)
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        dataframes.append(df)  # Add the DataFrame to the list

FileNotFoundError: [Errno 2] No such file or directory: '/content/Pollution/data_folder'

In [ ]:
pollution_df = pd.concat(dataframes, ignore_index=True)
pollution_df

In [ ]:
print ('shape', pollution_df.shape )

def unique_col(df):
  for column in df.columns:
    print (f'{column}: {df[column].nunique()}')

print(f'\nnumber of unigue value in each column')
unique_col(pollution_df)
print ('\n')
pollution_df.info()

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum() # Total missing values
    mis_val_percent = 100 * mis_val / len(df)  # Percentage of missing values
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})
    mis_val_table = mis_val_table.sort_values('% of Total Values', ascending=False)  # Sort the table by percentage of missing descending
    return mis_val_table

missing_values = missing_values_table(pollution_df)
display(missing_values.style.background_gradient(cmap='Blues'))

In [ ]:
pollution_df['Date']= pd.to_datetime(pollution_df['Date'], errors= 'raise',format= '%d/%m/%Y')
pollution_df['year']= pollution_df['Date'].dt.year
pollution_df['month']= pollution_df['Date'].dt.month
pollution_df['day']= pollution_df['Date'].dt.day
pollution_df['Month name']= pollution_df['Date'].dt.month_name()


In [ ]:
missing_values_per_city = pollution_df.groupby('City').apply(lambda x: x.isnull().sum())
missing_values_per_city

In [ ]:
plt.figure(figsize= (10,6))
sns.heatmap(missing_values_per_city)
plt.xlabel('Pollutant')
plt.show()

In [ ]:
pollution_df= pollution_df.sort_values(['City', 'Date'],ascending= [True, True])

In [ ]:
pollutants = ["PM2.5","PM10","NO","NO2","NOx","NH3","CO","SO2","O3",
              "Benzene","Toluene","Xylene", "AQI", "AQI_Bucket"]

pollution_df[pollutants] = pollution_df.groupby(['City', 'year'])[pollutants].ffill().bfill()
pollution_df.isna().sum()

In [ ]:

pollution_df.head(20)

In [ ]:
pollution_df.describe().T

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

pollutant_columns = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']

#Group by station and calculate mean pollutant concentrations
mean_pollutant_by_city =pollution_df.groupby('City')[pollutant_columns].mean()

#Find the top 5 stations for each pollutant
top_city = {}
for pollutant in pollutant_columns:
    top_city[pollutant] = mean_pollutant_by_city[pollutant].sort_values(ascending=False).head(5)

# Step 3: Plotting
fig, axes = plt.subplots(len(pollutant_columns), 1, figsize=(10, 20))

for i, pollutant in enumerate(pollutant_columns):
    axes[i].barh(top_city[pollutant].index, top_city[pollutant].values, color='red')
    axes[i].set_title(f'Top 5 Cities by {pollutant}')
    axes[i].set_xlabel(f'{pollutant}')
    axes[i].invert_yaxis()  # Highest values on top

plt.tight_layout()
plt.show()


In [ ]:
pollutants = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']
cities = pollution_df['City'].unique()

for city in cities:
    city_data = pollution_df[pollution_df['City'] == city]
    plt.figure(figsize=(10, 6))
    for pollutant in pollutants:
        plt.bar(pollutant, city_data[pollutant].mean(), label=pollutant)
    plt.title(f'Average Pollutant Levels at Station {city}')
    plt.xlabel('Pollutant')
    plt.ylabel('Average Concentration')
    plt.legend()
    plt.show()

In [ ]:
city_stats = pollution_df.groupby('City')[pollutants].mean().reset_index()

import plotly.express as px

# Melt the DataFrame to create a 'Pollutant' column
city_stats_melted = city_stats.melt(id_vars='City',
                               value_vars=pollutants,
                               var_name='Pollutant',
                               value_name='Average Concentration')

# Create the stacked bar chart
fig_cities= px.bar(city_stats_melted,
                     x='City',
                     y='Average Concentration',
                     color='Pollutant', # Use 'Pollutant' for color differentiation
                     title='Average Pollution Levels by City (Stacked)',
                     labels={'City': 'Cities', 'Average Concentration': 'Average Concentration'},
                     barmode='stack')

fig_cities.show()

In [ ]:
pollutants = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']
total_pollutant_concentration = pollution_df[pollutants].mean().sum()  # Calculates total concentration across all pollutants

# Calculate percentage for each pollutant
pollutant_percentages = [(pollution_df[pollutant].mean() / total_pollutant_concentration) * 100 for pollutant in pollutants]

# Create the pie chart
plt.figure(figsize=(8, 8))  # Adjust figure size if needed
perc_pol= px.pie(values= pollutant_percentages,names=pollutants, hole= 0.3,title= 'Percentage of Pollutants')
perc_pol.show()

In [ ]:
key_variables = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']

plt.figure(figsize=(15, 10))
for i, column in enumerate(key_variables, 1): # the subplot starts from index 1 and not from 0
  plt.subplot(4, 3, i)
  sns.histplot(pollution_df[column], kde=True, bins=30)
  plt.title(f'Distribution of {column}')
  plt.xlabel(column)
  plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
key_variables = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']

plt.figure(figsize=(15, 10))
for i, column in enumerate(key_variables, 1): # the subplot starts from index 1 and not from 0
  plt.subplot(4, 3, i)
  sns.boxplot(x= pollution_df[column])
  plt.title(f'Boxplot of {column}')
  plt.xlabel(column)
plt.tight_layout()
plt.show()

In [ ]:
# Defining a function to determine the season based on the month
def get_season(month):
  if 3 <= month <= 5:
    return 'Spring'
  elif 6 <= month <= 8:
    return 'Summer'
  elif 9 <= month <= 11:
    return 'Autumn'
  else:
    return 'Winter'

# Apply the function to the 'month' column and create a new 'Season' column
pollution_df['Season'] = pollution_df['month'].apply(get_season)

pollution_df.reset_index(inplace=True) #resetting the index to be true
pollution_df.head()


In [ ]:
# Group by season and calculate average pollutant concentrations
pollutants = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']
seasonal_avg = pollution_df.groupby('Season')[ pollutants].mean().reset_index()
# Melt the DataFrame for visualization
seasonal_avg_melted = seasonal_avg. melt(id_vars='Season', var_name='Pollutant', value_name='Average Concentration')
seasons= pollution_df['Season'].unique()
plt.figure(figsize= (15, 8))
for i, season in enumerate(seasons, 1):
  plt.subplot(2,2,i)
  val= seasonal_avg_melted[seasonal_avg_melted['Season']== season].sort_values('Average Concentration')
  sns.barplot(val, x='Average Concentration', y= 'Pollutant')
  plt.title(f'Average Pollutant Level in {season}')
plt.tight_layout()
plt.show()





In [ ]:
# relationship between AQI and each pollutant in each city
pollutants = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']
plt.figure(figsize= (15, 10))
for i, pollutant in enumerate(pollutants, 1):
  plt.subplot(4,3,i)
  sns.scatterplot(pollution_df,x= pollutant, y='AQI')
  plt.title('Vs AQI')
plt.tight_layout()
plt.show()



